In [ ]:
import mne
import pandas as pd 
import re #regex library to format fast to read into pd 
import os
import neurokit2 as nk 
import matplotlib.pyplot as plt
import feather 
import numpy as np
import platform

from files_in_out import getListOfFiles,GetFiles



In [ ]:
g_num= 'insert participant code'
eeg_format='.fif'
eeg_exp='tsk'
datafolder='preproc'

#files = getListOfFiles(datafolder,g_num)

files = GetFiles(datafolder,g_num=g_num,eeg_format=eeg_format)

taskfiles=files.taskfiles

taskfiles=[x for x in taskfiles if 'rec' not in x]
taskfiles=[x for x in taskfiles if '_n_'  in x]
taskfiles

In [ ]:
files.select_condition('n')
files.get_info(index=2,end_fix=-12,start_fix=16)

dir_tsk=files.current_file_dir
dir_tsk

In [ ]:
files.current_filename

In [ ]:
#%cd
def filter_list(list_,value):
    filter_list=[x for x in list_ if value in x]
    return filter_list[0]
    


#ica=mne.preprocessing.read_ica(filter_list(taskfiles,'_ica'))
ica=mne.preprocessing.read_ica(filter_list(taskfiles,'fast-ica'))
exp_epo=mne.read_epochs(filter_list(taskfiles,'exp'))
eog_epo=mne.read_epochs(filter_list(taskfiles,'eog'))
ecg_epo=mne.read_epochs(filter_list(taskfiles,'ecg'))


In [ ]:
eeg_format='.fif'
eeg_exp='tsk'
datafolder='raw'
condition='n'

#files = getListOfFiles(datafolder,g_num)

fif_files = GetFiles(datafolder,g_num=g_num,eeg_format=eeg_format)
fif_files.select_condition(condition)
fif_taskfiles=fif_files.condition_files
fif_taskfiles

In [ ]:
raw_dir=fif_taskfiles[0]
raw=mne.io.read_raw_fif(raw_dir,preload=True)


In [ ]:

exp_hep=exp_epo['xns']
exp_hep

In [ ]:
epo_id=exp_hep.event_id
epo_id

In [ ]:
labels=[k.split('/') for k,v in epo_id.items()]
labels=[x[:-1] for x in labels]
labels=[list(set(x).intersection(y)) for x in labels for y in labels]
labels=[list(i)for i in set(map(tuple,labels))]
labels_normal=[x for x in labels if 'normal' in x]
labels_normal.append(['RRCA','RRCU'])
labels_normal

In [ ]:
%matplotlib qt
eog_epo.average().plot_joint()

In [ ]:
eog_epo

In [ ]:

raw.plot(n_channels=68,events=eog_epo.events)

In [ ]:
eog_epo.plot()



In [ ]:
exp_epo['normal/vep']

In [ ]:
exp_epo['normal/vep/CU'].average().plot()


# Identify EOG components 

In [ ]:
#eog_ep=mne.preprocessing.create_eog_epochs(raw,ch_name=['C16'],thresh=40e-6)

In [ ]:
eog_epo

In [ ]:
eog_inds, eog_scores = ica.find_bads_eog(eog_epo,ch_name='C16')  # find via correlation

ica.plot_scores(eog_scores);  # look at r scores of components

In [ ]:
%matplotlib auto
plot_idx=np.arange(0,26)

ica.plot_components(inst=exp_epo['vep/normal'])

ica.plot_sources(inst=exp_epo['vep/normal'])

In [ ]:
ica.plot_sources(exp_epo['vep'].average())

In [ ]:
eog_inds

In [ ]:
eog_inds=[10,2]

In [ ]:

ica.plot_overlay(eog_epo.average(), exclude=eog_inds, show=False)
ica.plot_overlay(exp_epo['normal/vep'].average(), exclude=eog_inds, show=False)

# red -> before, black -> after. Yes! We remove quite a lot!


In [ ]:
#extra inds to test 
artefact_idx=[44,21]

ica.plot_overlay(eog_epo.average(), exclude=eog_inds+artefact_idx, show=False)
ica.plot_overlay(exp_epo['vep/normal'].average(), exclude=eog_inds+artefact_idx, show=False)

In [ ]:
print(ica.exclude)
print(eog_inds)
print(artefact_idx)

### Show selected bad component and investigate effect on data once removed

In [ ]:

ica.exclude=[]
ica.exclude.extend(eog_inds+artefact_idx)

In [ ]:
reconst_eog = exp_epo.copy()

ica.apply(reconst_eog)

In [ ]:
topomap_args={'outlines':'head'}
reconst_eog['normal/vep/CU'].average().plot_joint(topomap_args=topomap_args)

In [ ]:
reconst_eog['normal/vep/CA'].average().plot_image()

In [ ]:
reconst_eog_vep=reconst_eog['vep']
reconst_eog_vep.plot()

In [ ]:
reconst_eog_vep['normal/CU'].average().plot_joint()

# Heart artefact removal 
do the same thing for heart artefact to create data set without cardiac field artefact (nc)


In [ ]:
#plot and identify bad components
ecg_ica_indices, ecg_ica_scores = ica.find_bads_ecg(raw, method='correlation',
                                            threshold='auto')


ica.plot_scores(ecg_ica_scores)



In [ ]:
ecg_ica_indices

In [ ]:
ica.plot_sources(inst=ecg_epo.average())

In [ ]:

#ecg_ica_indices_3, ecg_ica_scores_3 = ica.find_bads_ecg(raw, method='correlation',
                                            #threshold='auto')

In [ ]:
print(ecg_ica_indices)


In [ ]:
ecg_ica_indices=[17,56]


In [ ]:
ica.plot_properties(inst=ecg_epo,picks=ecg_ica_indices)

In [ ]:
#manually identified ecg component = ICA14
ecg_remove=ecg_ica_indices

In [ ]:
#see effect of removing component on data
ica.plot_overlay(exp_epo['vep/dia'].average(), exclude=eog_inds+artefact_idx, show=False)
ica.plot_overlay(exp_epo['vep/dia'].average(), exclude=eog_inds+artefact_idx+ecg_remove, show=False)
ica.exclude=[]
ica.plot_overlay(exp_epo['hep/R2'].average(), exclude=eog_inds+artefact_idx, show=False)
ica.plot_overlay(exp_epo['hep/R2'].average(), exclude=eog_inds+artefact_idx+ecg_remove, show=False)

In [ ]:
#insert component to be excluded
ica.exclude=[]
ica.exclude.extend(eog_inds+ecg_remove+artefact_idx)
ica.exclude

In [ ]:
reconst_ecg = exp_epo.copy()

ica.apply(reconst_ecg)

In [ ]:
topomap_args={'outlines':'head'}
reconst_eog['vep/CU'].average().plot_joint(topomap_args=topomap_args)

In [ ]:
topomap_args={'outlines':'head'}
reconst_ecg['normal/vep/CA'].average().plot_joint(topomap_args=topomap_args)

# Save epochs output

In [ ]:
# save output without cardiac field artefact
type_sig='epochs'
file_end='nc_rec_epo.fif'

output_filename=files.out_filename(type_sig=type_sig,file_end=file_end)


reconst_ecg.save(output_filename,overwrite=True)

#save output including cardiac field artefact
type_sig='epochs'
file_end='cfa_epo.fif'

output_filename=files.out_filename(type_sig=type_sig,file_end=file_end)


reconst_eog.save(output_filename,overwrite=True)


In [ ]:
# save log 
file_end='ICA_log.txt'

output_filename=files.out_filename(type_sig=type_sig,file_end=file_end)



with open(output_filename,'w') as file:
    file.write(f'eog index={eog_inds}\n')
    file.write(f'artefact index={artefact_idx}\n')
    file.write(f'ecg index={ecg_ica_indices}\n')